In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
sns.set()
%matplotlib inline

# 전처리 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# 경고메세지 숨기기
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 중간에 잘리는 컬럼없이 100개까지 Print / 소수점은 5번째까지
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.5f}'.format

In [3]:
# 모델적용 라이브러리
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [4]:
#폰트 설정
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths = None, fontext = 'ttf')
font_list[:]

['C:\\Windows\\Fonts\\HARLOWSI.TTF',
 'C:\\Windows\\Fonts\\KUNSTLER.TTF',
 'C:\\Windows\\Fonts\\LEELAWDB.TTF',
 'C:\\Windows\\Fonts\\MAIAN.TTF',
 'C:\\Windows\\Fonts\\malgunbd.ttf',
 'C:\\Windows\\Fonts\\HPSimplified_BdIt.ttf',
 'C:\\Windows\\Fonts\\FRADMCN.TTF',
 'C:\\Windows\\Fonts\\NIAGENG.TTF',
 'C:\\Windows\\Fonts\\LSANSDI.TTF',
 'C:\\Windows\\Fonts\\HARNGTON.TTF',
 'C:\\Windows\\Fonts\\SCHLBKB.TTF',
 'C:\\Windows\\Fonts\\SCHLBKBI.TTF',
 'C:\\Windows\\Fonts\\PERTIBD.TTF',
 'C:\\Windows\\Fonts\\GOTHICB.TTF',
 'C:\\Windows\\Fonts\\mingliub.ttc',
 'C:\\Windows\\Fonts\\segoeuiz.ttf',
 'C:\\Windows\\Fonts\\GLSNECB.TTF',
 'C:\\Windows\\Fonts\\BOD_I.TTF',
 'C:\\Windows\\Fonts\\BOOKOSBI.TTF',
 'C:\\Windows\\Fonts\\cour.ttf',
 'C:\\Windows\\Fonts\\wingding.ttf',
 'C:\\Windows\\Fonts\\lucon.ttf',
 'C:\\Windows\\Fonts\\arial.ttf',
 'C:\\Windows\\Fonts\\H2GTRE.TTF',
 'C:\\Windows\\Fonts\\calibri.ttf',
 'C:\\Windows\\Fonts\\constanz.ttf',
 'C:\\Windows\\Fonts\\MTCORSVA.TTF',
 'C:\\Windows\\Fon

In [5]:
import platform 

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

## Making Plot

### 시간별 데이터값을 알아보기 위한 Plot
- 데이터, 컬럼, 시간컬럼(명)

In [1]:
def TimePlot(data,columns,timecolumn):
    plt.figure(figsize=(20,10))
    plt.title(columns)
    plt.plot(data[timecolumn], data[columns])
    plt.savefig("./plot/"+columns+"time.png")
    plt.show()

### 분포를 비교하기 위한 Plot
- 비교데이터1, 비교데이터2, 컬럼값

In [2]:
def Compare_Distribution(data_1,data_2,columns):
    plt.figure(figsize=(12,8))
    plt.subplot(121)
    plt.hist(data_1[columns])
    plt.title("{} Value (Compare_1)".format(columns))
    
    plt.subplot(122)
    plt.hist(data_2[columns])
    plt.title("{} Value (Compare_2)".format(columns))

### 0값을 대체하기 위한 함수
- 데이터, 컬럼, 값(변경하고싶은 값)

In [3]:
def change_zero(data,columns,values):
    for n,i in enumerate(columns):
        # 컬럼의 값이 0 이면 value[n]값으로 변경
        data[i] = data[i].apply(lambda x: values[n] if x == 0 else x)

### 이상치를 제거하기 위한 함수
- 데이터, 컬럼

In [5]:
def remove_iqr(data,column):
    # 데이터의 25% 75%값을 q1,q3에 저장
    q1,q3 = np.percentile(data[column],[25,75])
    # iqr값 저장
    iqr = q3-q1
    # Low Boundary 저장
    lower_bound = q1 - (iqr*1.5)
    # High Boundary 저장
    upper_bound = q3 + (iqr*1.5)
    
    # Low/High Boundary값을 제거하고 새롭게 DataFrame을 저장.
    remove_iqr_df = data[(data[column]>=lower_bound) & (data[column]<=upper_bound)]
    return(remove_iqr_df)

### PCA 그래프
- 데이터, 컬럼

In [1]:
def PCA_graph(data,column):
    fig = plt.figure(figsize = (8, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize=20)

    targets = [0,1]
    colors = ['g', 'b']
    for target, color in zip(targets,colors):
        indicesToKeep = data[column] == target
        ax.scatter(data.loc[indicesToKeep, 'Components_1']
                   , data.loc[indicesToKeep, 'Components_2']
                   , c = color)
    ax.legend(targets)
    ax.grid()

## 기타 전처리

In [ ]:
# 전처리 categrical data
df['variety'] = df['variety'].map({"Setosa":1, "Versicolor":2, "Virginica":2})